<a href="https://colab.research.google.com/github/yehocoh/ESG_PDF_TO_LLM/blob/main/yehonatan_cohen/using_alpaca_lora_bad_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Talk to Alpaca-LoRA


In [ ]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)

model = PeftModel.from_pretrained(model, "tloen/alpaca-lora-7b")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""For the follow sentence please classify it into just one of the following 5 categories based on its ESG related content.

                                            -'Quantitative Evidence': The sentence contains quantitative evidence that the company already has been taking actions to improve its ESG measures and already implemented the plans. Future plans do NOT count!
                                            -'Action Evidence': The sentence contains hard evidence that is not quantitative that the company already has been taking actions to improve its ESG measures and already implemented the plans. Future plans do NOT count! .
                                            -'Suggestive Evidence': The sentence doesn't contain hard evidence of taking actions to improve the company's ESG measures, but it suggests that such actions have been taken already and have been implemented. Future plans do NOT count!
                                            -'Intention': The sentence mentions a company's plans, intentions or commitments of taking actions in the future, without evidence of any actual actions yet. It include just future plans of the company to take actions related to ESG.
                                            -'Other': The sentence does not match any category.

                                            sentence :

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""For the follow sentence please classify it into just one of the following 5 categories based on its ESG related content.

                                            -'Quantitative Evidence': The sentence contains quantitative evidence that the company already has been taking actions to improve its ESG measures and already implemented the plans. Future plans do NOT count!
                                            -'Action Evidence': The sentence contains hard evidence that is not quantitative that the company already has been taking actions to improve its ESG measures and already implemented the plans. Future plans do NOT count! .
                                            -'Suggestive Evidence': The sentence doesn't contain hard evidence of taking actions to improve the company's ESG measures, but it suggests that such actions have been taken already and have been implemented. Future plans do NOT count!
                                            -'Intention': The sentence mentions a company's plans, intentions or commitments of taking actions in the future, without evidence of any actual actions yet. It include just future plans of the company to take actions related to ESG.
                                            -'Other': The sentence does not match any category.

                                            sentence :

### Instruction:
{instruction}

### Response:"""

In [ ]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)


def evaluate(instruction , input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        return output.split("### Response:")[1].strip()

In [ ]:
evaluate(input("Instruction: "))

Instruction: The collaboration could lead to capturing and storing up to 50 million metric tons of CO2 per year by 2030 and about 100 million metric tons by 2040.
Response: Action Evidence


In [ ]:
evaluate("Our PCI target for 2028 is 71 g CO2e/MJ, a > 5% decrease from 2016.")

'Action Evidence'

In [ ]:
evaluate("This methodology is expected to improve accuracy and build stakeholder confidence in data reliability.")

"'Action Evidence'"

# VALIDATION

In [ ]:
sentences = [
    "Our PCI target for 2028 is 71 g CO2e/MJ, a > 5% decrease from 2016.",
    "In collaboration with Pavilion Energy and QatarEnergy, we jointly developed a liquefied natural gas (LNG) carbonfootprinting methodology for delivered cargoes to help advance a standard for GHG product-level accounting.",
    "This methodology is expected to improve accuracy and build stakeholder confidence in data reliability.",
    "In early 2022, we announced a one-year pilot with Project Canary to enhance our ability to demonstrate transparency in how we are lowering methane emissions in our operations.",
    "Project Canary will use its comprehensive TrustWell Certification program to review and analyze the environmental and social performance aspects of individual wells and facilities in Colorado and Texas.",
    "The PCI metric, which includes Scope 1, 2 and 3, represents the carbon intensity across the full value chain associated with bringing products to market.",
    "Chevron's efforts to achieve our 2028 Upstream methane intensity target of 2.0 kg CO2e/boe include facility design, GHG reduction projects and exploring emerging technology to identify opportunities to further lower emissions.",
    "Our planned GHG reduction projects include opportunities to reduce venting, such as a nitrogen blanket system for the tank farm at Tengizchevroil (TCO), and to reduce flaring, which also reduces methane emissions.",
    "Our standard facility design in the Permian Basin includes methane emission controls, such as vapor recovery units at central tank batteries and pneumatic controllers that utilize compressed air instead of natural gas.",
    "We are committed to further improving methane detection and direct measurement through our global methane detection campaign, which focuses on scaling up proven and emerging detection technologies and modes of deployment, such as satellites, aircraft and drones.",
    "To date, we have completed campaigns in Argentina, the Denver-Julesburg Basin, the Gulf of Mexico, the Permian Basin and TCO.",
    "We are active participants in multiple partnerships and associations focused on methane emissions.",
    "Chevron is a founding member of the Collaboratory to Advance Methane Science (CAMS), a joint industry project to conduct peer-reviewed research around methane emissions.",
    "Recent CAMS projects include an aerial survey to understand sources in the Permian Basin and the first measurement study of methane emissions from LNG transport activities.",
    "Chevron is also a founding partner of the Environmental Partnership, an industry initiative aimed at accelerating the adoption of practices that reduce methane emissions.",
    "To date, companies in this initiative have conducted more than 770,000 leak detection surveys and replaced more than 27,000 pneumatic controllers with lower- or non-emitting technologies.",
    "Recently, the Environmental Partnership collaborated with several aerial survey technology providers, including the NASA Jet Propulsion Laboratory/University of Arizona, GHGSat, Bridger Photonics and Kairos Aerospace.",
    "We're investing to reduce methane emissions and flaring.",
    "Improving methane detection, rethinking facility designs, optimizing equipment and deploying new operational practices are a few examples of the projects underway to lower emissions.",
    "Completion operations flow back to permanent facilities that have equipment to capture entrained gas.",
    "Since 2011, standard facility designs have included compressed air for pneumatic controllers, which eliminate natural gas venting for that application.",
    "Standard designs for tank batteries and compressor stations include Vapor Recovery Units, which gather emissions that can be reused onsite or sold to third parties.",
    "Flaring intensity was 79% less than Permian Basin average in 2020 due to infrastructure planning to create gas-take away capacity.",
    "OurApologies for the cutoff in the previous message. Here's the continuation:",
    "Our Mid-Continent business unit has piloted eight advanced methane detection solutions since 2016 and selected an aerial laser-based methane scanning technology for broader deployment in the Permian.",
    "Aircraft-based solutions help us and other nearby operators cost-effectively screen assets for methane emissions across a wide geographic footprint in the Permian.",
    "Early in 2021, I was asked to serve as project manager for Chevron's methane detection campaign, which is tasked with deploying methane detection technology to cover 80% of our equity emissions.",
    "Our team works with business units across Chevron's asset classes to design tailored methane detection solutions to accommodate operational needs, policy concerns and regulatory protocols.",
    "We are experimenting with a variety of technologies to expand our methane detection capabilities, including deploying airborne sensors using satellites, aircraft and drones.",
    "In addition to working within Chevron's business units, we also collaborate with regulators, universities and other operators to evaluate equipment and share best practices.",
    "I'm especially proud of our work with the Gulf of Mexico business unit, where we became one of the first operators to deploy a methane detection solution offshore using drones.",
    "Offshore is especially challenging because ocean water can confuse detection technology and create false readings.",
    "In addition, platforms have layered equipment, which can make it difficult to identify the source of emissions from overhead sensors alone.",
    "To solve this challenge, we combined emissions detection equipment from our San Joaquin Valley business unit with drones used offshore and developed a new flight procedure to detect methane emissions from the platforms.",
    "After identification, the business units work to remove or mitigate the emissions source.",
    "Methane reduction is a global challenge, and there is not a one-size-fits-all solution.",
    "I am inspired by the level of expertise within our company and our culture of innovation and experimentation.",
    "Chevron endorses the World Bank's Zero Routine Flaring by 2030 initiative.",
    "We flare natural gas only when necessary for safety and operational purposes and in areas where pipelines and other alternatives for transporting gas do not exist.",
    "Our 2028 Upstream carbon intensity target includes a specific target for flaring intensity of 3.0 kg CO2e/boe, which is a 66% reduction from our 2016 baseline.",
    "Chevron endorsed the World Bank's Zero Routine Flaring by 2030 initiative, which brings together govern ments, oil companies and development institutions to cooperate to eliminate routine flaring by no later than 2030.",
    "Chevron is an active participant in the World Bank's Global Gas Flaring Reduction (GGFR) voluntary partnership.",
    "GGFR recently partnered with the Payne Institute for Public Policy at the Colorado School of Mines to develop a transparent web platform to support real-time mapping and tracking of global gas flaring data.",
    "Chevron supported this partnership through our membership in the Oil and Gas Climate Initiative.",
    "Future progress will require applying our world-class capabilities as we aim to deliver higher returns in a lower carbon world.",
    "Our capabilities, assets and customers are distinct advantages.",
    "Chevron Technology Ventures targets external innovation and transformational technology.",
    "The Chevron Technical Center develops and deploys technology across the entire business, including integrating low-carbon technology into our operations.",
    "In 2021, we formed Chevron New Energies, a new organization dedicated to growing hydrogen, CCUS, offsets and other emerging energies.",
    "Chevron Strategy and Sustainability continues to steward the company's long-term strategy by integrating climate change, energy transition and other sustainability themes into macroeconomic forecasting, supply-and-demand forecasting, price forecasting, portfolio modelingApologies for the cutoff again. Here's the continuation:",
    "Chevron Strategy and Sustainability continues to steward the company's long-term strategy by integrating climate change, energy transition and other sustainability themes into macroeconomic forecasting, supply-and-demand forecasting, price forecasting, portfolio modeling and competitor intelligence.",
    "Chevron and more than 10 industry partners have agreed to support large-scale deployment of carbon capture and storage (CCS) to help decarbonize industrial facilities in Houston, Texas, one of the largest concentrated sources in the United States.",
    "The organizations are considering using CCS technology at facilities that generate electricity and manufacture everyday products such as plastics, motor fuels and packaging.",
    "The collaboration could lead to capturing and storing up to 50 million metric tons of CO2 per year by 2030 and about 100 million metric tons by 2040."
]

labels = [
    "Intention",
    "Action Evidence",
    "Suggestive Evidence",
    "Intention",
    "Other",
    "Other",
    "Intention",
    "Intention",
    "Action Evidence",
    "Intention",
    "Quantitative Evidence",
    "Other",
    "Action Evidence",
    "Action Evidence",
    "Action Evidence",
    "Quantitative Evidence",
    "Other",
    "Intention",
    "Action Evidence",
    "Action Evidence",
    "Action Evidence",
    "Action Evidence",
    "Quantitative Evidence",
    "Quantitative Evidence",
    "Other",
    "Other",
    "Action Evidence",
    "Other",
    "Other",
    "Other",
    "Action Evidence",
    "Other",
    "Other",
    "Other",
    "Other",
    "Intention",
    "Other",
    "Intention",
    "Intention",
    "Action Evidence",
    "Other",
    "Other",
    "Other",
    "Other",
    "Other",
    "Other",
    "Other",
    "Intention",
    "Other",
    "Other",
    "Other",
    "Other",
    "Other",
    "Intention"
]


In [ ]:
def class_row_in_text_(txt) :
      response = evaluate(txt)
      return [response]

In [ ]:
all_responses_ = []
for i_ in sentences :
    responses = class_row_in_text_(i_)
    all_responses_ = all_responses_ + responses

In [ ]:
import pandas as pd

df_ = pd.DataFrame(list(zip(labels, all_responses_)), columns=['labels', 'response'])
df_


,labels,response
0,Intention,Action Evidence
1,Action Evidence,Action Evidence
2,Suggestive Evidence,'Action Evidence'
3,Intention,Action Evidence
4,Other,Action Evidence
5,Other,'Quantitative Evidence'
6,Intention,Action Evidence
7,Intention,Action Evidence
8,Action Evidence,Action Evidence
9,Intention,Action Evidence
